# Bounce model
#### This code solves the linearly time-dependent Bernoulli model using log-likelihood maximization

In [1]:
import pandas as pd
import numpy as np
from scipy import optimize
from scipy import stats as st

In [2]:
# maximum time allowed (time when data was gathered) in fractional time
t0 = 2018.87

input_file = "AND_bounced_data.xlsx"

In [3]:
bounced_data_original = pd.read_excel(input_file)

# compute difference time between bounce time and maximum time allowed
bounced_data_original.Time = t0 - bounced_data_original.Time

In [4]:
# function to be minimized: linearly time-dependent Bernoulli
def log_likelihood_function(alpha):
    
    # compute log-likelihood of bounce data
    p_vector = bounced_data.Time[bounced_data.Bounced] * alpha
    logsum_p = np.log10(p_vector).sum()

    # compute log-likelihood of not-bounced data
    q_vector = 1 - bounced_data.Time[~bounced_data.Bounced] * alpha
    logsum_q = np.log10(q_vector).sum()

    # sum both log-likelihoods
    total_logsum = logsum_p + logsum_q
    
    # function will be minimized, hence return the likelihood times -1
    return -total_logsum

In [5]:
# find log-likelihood maximum
# note that bounds have to be limited because higher values would lead to negative probabilities
bounced_data = bounced_data_original.copy()
solution = optimize.minimize_scalar(log_likelihood_function, bounds=[0,0.05], method='bounded')

# output solution
print(solution)

     fun: 58.37626102019652
 message: 'Solution found.'
    nfev: 9
  status: 0
 success: True
       x: 0.021027097065903593


In [6]:
# bootstrap
solutions = []

iterations = 1000

for i in range(iterations):
    bounced_data = bounced_data_original.sample(frac=1, axis=0, replace=True)
    solution = optimize.minimize_scalar(log_likelihood_function, bounds=[0,0.05], method='bounded')
    solutions.append(solution.x)

In [7]:
# calculate values within the 95% CI
sorted_solutions = sorted(solutions)
solutions_upper_bound = sorted_solutions[int(len(sorted_solutions)*0.975)]
solutions_lower_bound = sorted_solutions[int(len(sorted_solutions)*0.025)]

print("95% confidence interval: [" + str(solutions_upper_bound) + ", " + str(solutions_lower_bound) + "]")

95% confidence interval: [0.026137655463274085, 0.01618178982832137]
